Evaluate Retrieval Impact
===

Questions:
 - Does retrieval matter much?
 - Does it matter if the retrieval is relevant to the user's query?



In [117]:
import json
import os
import pickle
import random
import re
import string as string_utils
import time
from collections import Counter
from pathlib import Path
from pprint import pprint

import dotenv
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import openai
import pandas as pd
import scipy
import sklearn.metrics
import tiktoken
from tqdm import tqdm

from llm_math_education import prompt_utils, retrieval, retrieval_strategies
from llm_math_education.prompts import mathqa as mathqa_prompts

In [2]:
dotenv.load_dotenv("../.env")

True

In [3]:
os.environ["OPENAI_API_KEY"][:3]

'sk-'

In [4]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [5]:
data_dir = Path("../data")
assert data_dir.exists()
with open(data_dir / "derived" / "rori_lessons.json") as infile:
    df = pd.read_json(infile)
df.shape

(67, 18)

### Load query data

In [6]:
mn_general_student_queries_filepath = data_dir / "app_data" / "mn_general_student_queries.csv"
query_df = pd.read_csv(mn_general_student_queries_filepath)
student_queries = [
    row.post_content.strip().replace("[Continued:]", "\n")
    for row in query_df.sample(frac=1, random_state=87896).itertuples()
    if row.is_respondable_query == "general"
]
len(student_queries)

51

In [7]:
student_queries[0]

'Can I get the steps for factoring quadratics'

In [8]:
student_queries

['Can I get the steps for factoring quadratics',
 'I need a standard way of recognizing polynomials and their degrees, please help me figure\nit out.',
 'how do i solve square root functions',
 'Can someone explain like interior and exterior angles? thanks',
 'How do you find the radius?',
 'What is vertex form and how do you solve for it?',
 'How do I multiply fractions???????',
 'Can someone help me with what an irrational number is vs a rational number',
 'How do you find the parent function of a graph?',
 'how do you know if a graph is an absolute value graph?',
 'I always forget the difference between commutative and associative. Does anybody know a way to make me remember?',
 'What is a function notation?',
 'I know that sin A=opposite/hypotenuse, cos A=adjacent/hypotenuse, and tan A=opposite/adjacent. Is there any other ratios like those?',
 "what are regression and median-fit lines?\n\n I really don't get them.(it) (that)",
 'after you distribute a negative does it disappear or

### Load dbs

In [9]:
app_embedding_dir = data_dir / "app_data"
recipe_embedding_dir = data_dir / "derived" / "embeddings"

In [10]:
rori_db = retrieval.RetrievalDb(app_embedding_dir, "rori_microlesson", "db_string")
openstax_db = retrieval.RetrievalDb(app_embedding_dir, "openstax_subsection", "db_string")
recipe_db = retrieval.RetrievalDb(recipe_embedding_dir, "recipenlg", "db_string")

In [11]:
rori_microlesson_db_info = retrieval.DbInfo(
    rori_db,
    max_tokens=1000,
    # prefix="Here is some lesson content that might be relevant:\n",
    prefix="Connect the student's question to this lesson content:\n",
)
openstax_subsection_db_info = retrieval.DbInfo(
    openstax_db,
    max_tokens=600,
    # prefix="Here are some excerpts from a math textbook. If they are relevant to the question, feel free to use language or examples from these excerpts:\n",
    prefix="Use language from these math textbook excerpts in your response:\n",
)
recipenlg_db_info = retrieval.DbInfo(
    recipe_db,
    max_tokens=2000,
    # prefix="Here are some step-by-step descriptions. If they are relevant to the question, feel free to use language or examples from these excerpts:\n",
    prefix="Use language from these step-by-step descriptions in your response:\n",
)
math_strategy = retrieval_strategies.MappedEmbeddingRetrievalStrategy(
    {
        "rori_microlesson_texts": rori_microlesson_db_info,
        "openstax_subsection_texts": openstax_subsection_db_info,
    },
)
nomath_strategy = retrieval_strategies.MappedEmbeddingRetrievalStrategy(
    {
        "rori_microlesson_texts": recipenlg_db_info,
    },
)
noretrieval_strategy = retrieval_strategies.NoRetrievalStrategy()

#### Exploration of conditioning on retrieval

In [12]:
recipenlg_db_info = retrieval.DbInfo(
    recipe_db,
    max_tokens=2000,
    max_texts=1,
    prefix="Model your response off the following recipe:\n",
)
nomath_strategy = retrieval_strategies.MappedEmbeddingRetrievalStrategy(
    {
        "recipe_texts": recipenlg_db_info,
    },
)

In [13]:
intro_prompt_messages = [
    {
        "role": "system",
        "content": """You are going to act as a mathematics tutor for a 13 year old student who is in grade 8 or 9.
This student lives in Ghana or Nigeria.
You will be encouraging and factual.

{recipe_texts}

Prefer simple, short responses.
If the student says something inappropriate or off topic you will say you can only focus on mathematics and ask them if they have any math-related follow-up questions.
""",
    },
]
for user_query in tqdm(student_queries, desc="Generating responses"):
    prompt_manager = prompt_utils.PromptManager()
    prompt_manager.set_intro_messages(intro_prompt_messages)
    prompt_manager.set_retrieval_strategy(nomath_strategy)
    messages = prompt_manager.build_query(user_query)
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        request_timeout=20,
    )
    assistant_message = completion["choices"][0]["message"]["content"]
    break
print(assistant_message)

Generating responses:   0%|                                                                                                     | 0/51 [00:08<?, ?it/s]

Of course! Factoring quadratics involves breaking down a quadratic equation into its factors. Here are the steps:

1. Start with a quadratic equation in the form of ax^2 + bx + c = 0.
2. Look for common factors in the three terms, such as a number that divides evenly into all of them.
3. If there are common factors, factor them out from each term.
4. Rewrite the equation with the factored terms.
5. Look for two binomials that multiply together to give you the factored expression.
6. Write the two binomials and set them equal to zero.
7. Solve each binomial equation separately.
8. The solutions you find will be the factors of the quadratic equation.

I hope these steps help you with factoring quadratics! Do you have any other math-related questions?


In [14]:
recipenlg_db_info = retrieval.DbInfo(
    recipe_db,
    max_tokens=2000,
    max_texts=1,
    prefix="",
)
nomath_strategy = retrieval_strategies.MappedEmbeddingRetrievalStrategy(
    {
        "recipe_texts": recipenlg_db_info,
    },
)
intro_prompt_messages = [
    {
        "role": "system",
        "content": """You are going to act as a mathematics tutor for a 13 year old student who is in grade 8 or 9.
This student lives in Ghana or Nigeria.
You will be encouraging and factual.
Prefer simple, short responses.
If the student says something inappropriate or off topic you will say you can only focus on mathematics and ask them if they have any math-related follow-up questions.""",
    },
    {
        "role": "user",
        "content": """Answer the following question: {user_query}

Use the following recipe as an analogy in your response, creating a corresponding mathematical operation for each step in the recipe:
{recipe_texts}""",
    },
]
for user_query in tqdm(random.sample(student_queries, k=2), desc="Generating responses"):
    prompt_manager = prompt_utils.PromptManager()
    prompt_manager.set_intro_messages(intro_prompt_messages)
    prompt_manager.set_retrieval_strategy(nomath_strategy)
    messages = prompt_manager.build_query(user_query)
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        request_timeout=20,
    )
    assistant_message = completion["choices"][0]["message"]["content"]
    print(len(messages))
    for message in messages:
        print(message["role"])
        print(message["content"] + "\n")
    print("RESPONSE")
    print(assistant_message)
    print("\n\n\n")

Generating responses:  50%|███████████████████████████████████████████████                                               | 1/2 [00:12<00:12, 12.35s/it]

2
system
You are going to act as a mathematics tutor for a 13 year old student who is in grade 8 or 9.
This student lives in Ghana or Nigeria.
You will be encouraging and factual.
Prefer simple, short responses.
If the student says something inappropriate or off topic you will say you can only focus on mathematics and ask them if they have any math-related follow-up questions.

user
Answer the following question: Can I get the steps for factoring quadratics

Use the following recipe as an analogy in your response, creating a corresponding mathematical operation for each step in the recipe:
Blonde Brownies recipe:  - Grease a 13 x 9 x 2-inch baking pan.  - Combine flour, baking powder and salt.  - Melt butter; remove from heat.  - Stir in sugar. Add eggs and vanilla.  - Stir until blended.  - Stir dry ingredients and walnuts into sugar mixture.  - Spread in pan.  - Bake in a 350° oven for 20 to 25 minutes.  - Cut into bars while warm.  - Makes 48 bars.

RESPONSE
Yes, I can help you with

Generating responses: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:23<00:00, 11.72s/it]

2
system
You are going to act as a mathematics tutor for a 13 year old student who is in grade 8 or 9.
This student lives in Ghana or Nigeria.
You will be encouraging and factual.
Prefer simple, short responses.
If the student says something inappropriate or off topic you will say you can only focus on mathematics and ask them if they have any math-related follow-up questions.

user
Answer the following question: I always forget the difference between commutative and associative. Does anybody know a way to make me remember?

Use the following recipe as an analogy in your response, creating a corresponding mathematical operation for each step in the recipe:
Pancakes recipe:  - Mix well.  - Pour on hot griddle until brown.

RESPONSE
Sure, I can help you remember the difference between commutative and associative using a pancake recipe as an analogy.

In the pancake recipe, "mixing well" is similar to the commutative property in mathematics. This property states that the order of the numb

In [18]:
rori_microlesson_db_info = retrieval.DbInfo(
    rori_db,
    max_tokens=1000,
    max_texts=3,
    # prefix="Here is some lesson content that might be relevant:\n",
    prefix="If relevant, refer to the explanation and example in these three lessons:\n",
)
openstax_subsection_db_info = retrieval.DbInfo(
    openstax_db,
    max_tokens=3000,
    max_texts=1,
    prefix="",
    use_parent_text=True,
    parent_group_cols=["chapter", "section"],
    parent_sort_cols=["index"],
)
math_strategy = retrieval_strategies.MappedEmbeddingRetrievalStrategy(
    {
        "rori_microlesson_texts": rori_microlesson_db_info,
        "openstax_subsection_texts": openstax_subsection_db_info,
    },
)
intro_prompt_messages = [
    {
        "role": "system",
        "content": """You are going to act as a mathematics tutor for a 13 year old student who is in grade 8 or 9.
This student lives in Ghana or Nigeria.
You will be encouraging and factual.
Prefer simple, short responses.
If the student says something inappropriate or off topic you will say you can only focus on mathematics and ask them if they have any math-related follow-up questions.""",
    },
    {
        "role": "user",
        "content": """Answer the following question: {user_query}

Reference content from this textbook section in your response:
{recipe_texts}""",
    },
]
for user_query in tqdm(random.sample(student_queries, k=2), desc="Generating responses"):
    prompt_manager = prompt_utils.PromptManager()
    prompt_manager.set_intro_messages(intro_prompt_messages)
    prompt_manager.set_retrieval_strategy(math_strategy)
    messages = prompt_manager.build_query(user_query)
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        request_timeout=20,
    )
    assistant_message = completion["choices"][0]["message"]["content"]
    print(len(messages))
    for message in messages:
        print(message["role"])
        print(message["content"] + "\n")
    print("RESPONSE")
    print(assistant_message)
    print("\n\n\n")

Generating responses:  50%|█████████████████████████████████████████████                                             | 1/2 [00:06<00:06,  6.86s/it]

2
system
You are going to act as a mathematics tutor for a 13 year old student who is in grade 8 or 9.
This student lives in Ghana or Nigeria.
You will be encouraging and factual.
Prefer simple, short responses.
If the student says something inappropriate or off topic you will say you can only focus on mathematics and ask them if they have any math-related follow-up questions.

user
Answer the following question: How do i graph an inequality with a less than or equal to sign?

Reference content from this textbook section in your response:


RESPONSE
To graph an inequality with a "less than or equal to" sign (≤), you can follow these steps:

1. Start by treating the inequality as if it were an equation. Let's assume you have an inequality like x ≤ 3.

2. Draw a number line and mark the point where the value is 3.

3. Since the inequality includes "less than or equal to," we need to indicate that the value can also be equal to 3. In this case, you can represent it with a closed circle (●

Generating responses: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:16<00:00,  8.33s/it]

2
system
You are going to act as a mathematics tutor for a 13 year old student who is in grade 8 or 9.
This student lives in Ghana or Nigeria.
You will be encouraging and factual.
Prefer simple, short responses.
If the student says something inappropriate or off topic you will say you can only focus on mathematics and ask them if they have any math-related follow-up questions.

user
Answer the following question: wait... so if im using pemdas how does that work with fractions ?

Reference content from this textbook section in your response:


RESPONSE
When it comes to using PEMDAS (also known as BODMAS) with fractions, it's a little different but still follows the same order. Let me break it down for you:

P - Parentheses: If you have any parentheses in the expression, start solving within them first. For example, if you have (1/2 + 3/4) * 2, you would solve within the parentheses first to get 5/4. 

E - Exponents: If there are any exponents, simplify them next. For example, if you hav

In [11]:
rori_microlesson_db_info = retrieval.DbInfo(
    rori_db,
    max_tokens=1000,
    max_texts=3,
    # prefix="Here is some lesson content that might be relevant:\n",
    prefix="If relevant, refer to the explanation and example in these three lessons:\n",
)
openstax_subsection_db_info = retrieval.DbInfo(
    openstax_db,
    max_tokens=1000,
    max_texts=3,
    # prefix="Here are some excerpts from a math textbook. If they are relevant to the question, feel free to use language or examples from these excerpts:\n",
    prefix="Use language from these math textbook excerpts in your response:\n",
)
math_strategy = retrieval_strategies.MappedEmbeddingRetrievalStrategy(
    {
        "rori_microlesson_texts": rori_microlesson_db_info,
        "openstax_subsection_texts": "",  # openstax_subsection_db_info,
    },
)

In [17]:
for user_query in tqdm(student_queries, desc="Generating responses"):
    intro_prompt_messages = [
        {
            "role": "system",
            "content": """You are going to act as a mathematics tutor for a 13 year old student who is in grade 8 or 9.
This student lives in Ghana or Nigeria.
You will be encouraging and factual.

{openstax_subsection_texts}

Prefer simple, short responses.
If the student says something inappropriate or off topic you will say you can only focus on mathematics and ask them if they have any math-related follow-up questions.
""",
        },
        {
            "role": "user",
            "content": user_query
            + """

{rori_microlesson_texts}
""",
        },
    ]

    prompt_manager = prompt_utils.PromptManager()
    prompt_manager.set_intro_messages(intro_prompt_messages)
    prompt_manager.set_retrieval_strategy(math_strategy)
    messages = prompt_manager.build_query(None, query_for_retrieval_context=user_query)
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        request_timeout=20,
    )
    assistant_message = completion["choices"][0]["message"]["content"]
    break
print(len(messages))
for message in messages:
    print(message["content"] + "\n")
print("RESPONSE")
print(assistant_message)

Generating responses:   0%|                                                                                                 | 0/51 [00:02<?, ?it/s]

2
You are going to act as a mathematics tutor for a 13 year old student who is in grade 8 or 9.
This student lives in Ghana or Nigeria.
You will be encouraging and factual.



Prefer simple, short responses.
If the student says something inappropriate or off topic you will say you can only focus on mathematics and ask them if they have any math-related follow-up questions.


Can I get the steps for factoring quadratics

If relevant, refer to the explanation and example in these three lessons:
It is good to have you here! Let's learn how to find the square root of numbers.  Squareroot of a number is a number when multiplied by itself gives the original number. Square root is denoted by √. Note that square roots exist only for positive numbers.  Example: Find the square root of the given number Ques:  √196 Ans: 14 Hint: 14 multiplied by 14 gives the number 196.   It's time for you to pratice. Shall we get started?  Write yes to proceed.
In today's lesson, we are going to learn to identif

In [14]:
pprint(prompt_manager.recent_slot_fill_dict)

[{'openstax_subsection_texts': ''},
 {'rori_microlesson_texts': 'If relevant, refer to the explanation and example '
                            'in these three lessons:\n'
                            "It is good to have you here! Let's learn how to "
                            'find the square root of numbers.  Squareroot of a '
                            'number is a number when multiplied by itself '
                            'gives the original number. Square root is denoted '
                            'by √. Note that square roots exist only for '
                            'positive numbers.  Example: Find the square root '
                            'of the given number Ques:  √196 Ans: 14 Hint: 14 '
                            "multiplied by 14 gives the number 196.   It's "
                            'time for you to pratice. Shall we get started?  '
                            'Write yes to proceed.\n'
                            "In today's lesson, we are going to l

#### logit_bias for high-faithfulness retrieval

In [15]:
rori_microlesson_db_info = retrieval.DbInfo(
    rori_db,
    max_tokens=1000,
    max_texts=3,
    # prefix="Here is some lesson content that might be relevant:\n",
    prefix="If relevant, refer to the explanation and example in these three lessons:\n",
)
openstax_subsection_db_info = retrieval.DbInfo(
    openstax_db,
    max_tokens=3000,
    max_texts=1,
    prefix="",
    use_parent_text=True,
    parent_group_cols=["chapter", "section"],
    parent_sort_cols=["index"],
)
math_strategy = retrieval_strategies.MappedEmbeddingRetrievalStrategy(
    {
        "rori_microlesson_texts": rori_microlesson_db_info,
        "openstax_subsection_texts": openstax_subsection_db_info,
    },
)
intro_prompt_messages = [
    {
        "role": "system",
        "content": """You are going to act as a mathematics tutor for a 13 year old student who is in grade 8 or 9.
This student lives in Ghana or Nigeria.
You will be encouraging and factual.
Prefer simple, short responses.
If the student says something inappropriate or off topic you will say you can only focus on mathematics and ask them if they have any math-related follow-up questions.""",
    },
    {
        "role": "user",
        "content": """Answer the following question: {user_query}

Reference content from this textbook section in your response:
{openstax_subsection_texts}""",
    },
]
for user_query in tqdm(random.sample(student_queries, k=2), desc="Generating responses"):
    prompt_manager = prompt_utils.PromptManager()
    prompt_manager.set_intro_messages(intro_prompt_messages)
    prompt_manager.set_retrieval_strategy(math_strategy)
    messages = prompt_manager.build_query(user_query)
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        request_timeout=20,
    )
    assistant_message = completion["choices"][0]["message"]["content"]
    print(len(messages))
    for message in messages:
        print(message["role"])
        print(message["content"] + "\n")
    print("RESPONSE")
    print(assistant_message)
    print("\n\n\n")

Generating responses:  50%|███████████████████████████████████████████████                                               | 1/2 [00:07<00:07,  7.66s/it]

2
system
You are going to act as a mathematics tutor for a 13 year old student who is in grade 8 or 9.
This student lives in Ghana or Nigeria.
You will be encouraging and factual.
Prefer simple, short responses.
If the student says something inappropriate or off topic you will say you can only focus on mathematics and ask them if they have any math-related follow-up questions.

user
Answer the following question: How do you find the radius?

Reference content from this textbook section in your response:
Convert Fractions to Decimals: In Decimals, we learned to convert decimals to fractions. Now we will do the reverse—convert fractions to decimals. Remember that the fraction bar indicates division. So 45 can be written 4÷5 or 54. This means that we can convert a fraction to a decimal by treating it as a division problem.Repeating Decimals: So far, in all the examples converting fractions to decimals the division resulted in a remainder of zero. This is not always the case. Let’s see wha

Generating responses: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.14s/it]

2
system
You are going to act as a mathematics tutor for a 13 year old student who is in grade 8 or 9.
This student lives in Ghana or Nigeria.
You will be encouraging and factual.
Prefer simple, short responses.
If the student says something inappropriate or off topic you will say you can only focus on mathematics and ask them if they have any math-related follow-up questions.

user
Answer the following question: what is the difference between communtative property of addition and addition property of equality

Reference content from this textbook section in your response:
Use the Commutative and Associative Properties: Think about adding two numbers, such as 5 and 3.  The results are the same. 5+3=3+5  Notice, the order in which we add does not matter. The same is true when multiplying 5 and 3.  Again, the results are the same! 5·3=3·5. The order in which we multiply does not matter.  These examples illustrate the commutative properties of addition and multiplication.  The commutative

In [31]:
stopword_df = pd.read_csv(data_dir / "derived" / "dolma_most_frequent_words.csv")
stopword_df.head(5)

,word,pct_wiki,rank_wiki,pct_gutenberg,rank_gutenberg,pct_s2,rank_s2
0,the,0.062537,1.0,0.060334,1.0,0.059086,1.0
1,of,0.034112,2.0,0.035786,2.0,0.038057,2.0
2,and,0.029596,3.0,0.030370,3.0,0.029543,3.0
3,in,0.025598,4.0,0.017187,6.0,0.021399,4.0
4,to,0.020756,5.0,0.025746,4.0,0.019590,5.0


In [58]:
n = 10000
sdf = stopword_df[(stopword_df.rank_wiki <= n) & (stopword_df.rank_gutenberg <= n) & (stopword_df.rank_s2 <= n)]
n = 1000
sdf = sdf[(sdf.rank_wiki <= n) | (sdf.rank_gutenberg <= n) | (sdf.rank_s2 <= n)].copy()
len(sdf)

1518

In [59]:
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
sdf["token_counts"] = [len(tokenizer.encode(row.word)) for row in sdf.itertuples()]
# sdf[sdf.token_counts > 1]
(sdf.token_counts == 1).sum()

1269

In [60]:
# a reminder: tokens are case-sensitive!
tokenizer.encode("The"), tokenizer.encode("the")

([791], [1820])

In [61]:
sdf = sdf[sdf.token_counts == 1].copy()
len(sdf)

1269

In [62]:
", ".join(
    sdf[(sdf.rank_gutenberg <= 1000) & (sdf.rank_s2 <= 1000) & (sdf.rank_wiki <= 1000)]
    .sort_values(by="rank_gutenberg")
    .word
)

'the, of, and, to, a, in, that, was, I, his, is, it, with, as, for, had, be, The, not, on, by, at, which, have, from, but, this, were, all, or, they, are, their, one, an, so, been, we, them, who, would, no, will, when, there, out, more, It, up, if, into, has, its, some, could, what, time, very, than, do, other, upon, about, any, only, now, then, A, In, made, like, two, may, did, can, should, these, before, over, such, see, must, first, much, after, good, most, where, those, way, well, day, being, through, long, many, This, life, They, make, same, even, without, every, There, found, under, how, last, people, while, still, also, take, years, If, work, against, three, another, right, left, part, When, get, between, each, few, give, As, called, new, For, because, To, side, both, find, At, seen, days, having, water, among, better, however, small, end, almost, until, taken, given, On, large, light, others, set, best, present, whose, less, does, power, used, rather, death, since, next, 1, ord

In [63]:
", ".join(sdf[sdf.rank_gutenberg <= 1000].sort_values(by="rank_gutenberg").word)

'the, of, and, to, a, in, that, was, I, his, is, he, it, with, as, for, had, be, The, not, on, by, at, you, which, her, have, from, but, this, were, him, all, or, they, are, she, their, one, an, my, so, me, been, we, them, who, said, would, He, no, will, when, there, out, more, It, up, if, into, has, its, And, some, But, could, what, time, very, than, do, man, other, our, upon, about, any, only, your, little, now, then, A, In, made, like, two, may, did, great, can, should, these, before, over, such, us, see, must, first, much, after, down, know, men, good, most, where, She, old, those, own, way, never, came, well, day, being, through, long, come, many, This, life, They, go, back, make, same, might, too, shall, You, even, without, here, say, again, every, There, found, We, under, how, am, think, went, last, place, people, away, while, still, also, thought, take, hand, just, years, eyes, If, work, though, off, against, three, another, right, yet, left, What, part, young, When, get, once,

In [64]:
# sdf["token"] = [tokenizer.encode(row.word)[0] for row in sdf.itertuples()]

In [103]:
stopword_tokens = set()
stopwords = (
    list(sdf.word)
    + list(sdf.word.map(str.lower))
    + list(sdf.word.map(str.upper))
    + list(sdf.word.map(str.capitalize))
    + list(string_utils.whitespace)
    + list(string_utils.punctuation)
)
for word in stopwords:
    for char in string_utils.whitespace + string_utils.punctuation:
        for string in [word, char + word, word + char]:
            tokens = tokenizer.encode(string)
            if len(tokens) == 1:
                stopword_tokens.add(tokens[0])
len(stopword_tokens)

11110

In [140]:
d = {"stopword_words": list(sdf.word), "stopword_tokens": list(stopword_tokens)}
with open(data_dir / "derived" / "dolma_stopwords.json", "w") as outfile:
    json.dump(d, outfile)
with open(Path("..") / "src" / "llm_math_education" / "resources" / "dolma_stopwords.json", "w") as outfile:
    json.dump(d, outfile)

In [67]:
rori_microlesson_db_info = retrieval.DbInfo(
    rori_db,
    max_tokens=1000,
    max_texts=3,
    # prefix="Here is some lesson content that might be relevant:\n",
    prefix="If relevant, refer to the explanation and example in these three lessons:\n",
)
openstax_subsection_db_info = retrieval.DbInfo(
    openstax_db,
    max_tokens=3000,
    max_texts=1,
    prefix="",
    use_parent_text=True,
    parent_group_cols=["chapter", "section"],
    parent_sort_cols=["index"],
)
math_strategy = retrieval_strategies.MappedEmbeddingRetrievalStrategy(
    {
        "rori_microlesson_texts": rori_microlesson_db_info,
        "openstax_subsection_texts": openstax_subsection_db_info,
    },
)
intro_prompt_messages = [
    {
        "role": "system",
        "content": """You are going to act as a mathematics tutor for a 13 year old student who is in grade 8 or 9.
This student lives in Ghana or Nigeria.
You will be encouraging and factual.
Prefer simple, short responses.
If the student says something inappropriate or off topic you will say you can only focus on mathematics and ask them if they have any math-related follow-up questions.""",
    },
    {
        "role": "user",
        "content": """Answer the following question: {user_query}

Reference content from this textbook section in your response:
{openstax_subsection_texts}""",
    },
]
for user_query in tqdm(random.sample(student_queries, k=2), desc="Generating responses"):
    prompt_manager = prompt_utils.PromptManager()
    prompt_manager.set_intro_messages(intro_prompt_messages)
    prompt_manager.set_retrieval_strategy(math_strategy)
    messages = prompt_manager.build_query(user_query)
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        request_timeout=20,
    )
    assistant_message = completion["choices"][0]["message"]["content"]
    print(len(messages))
    for message in messages:
        print(message["role"])
        print(message["content"] + "\n")
    print("RESPONSE")
    print(assistant_message)
    print("\n\n\n")
    break

Generating responses:   0%|                                                                                                      | 0/2 [00:09<?, ?it/s]

2
system
You are going to act as a mathematics tutor for a 13 year old student who is in grade 8 or 9.
This student lives in Ghana or Nigeria.
You will be encouraging and factual.
Prefer simple, short responses.
If the student says something inappropriate or off topic you will say you can only focus on mathematics and ask them if they have any math-related follow-up questions.

user
Answer the following question: wait... so if im using pemdas how does that work with fractions ?

Reference content from this textbook section in your response:
Find the Least Common Denominator: In the previous section, we explained how to add and subtract fractions with a common denominator. But how can we add and subtract fractions with unlike denominators?  Let’s think about coins again. Can you add one quarter and one dime? You could say there are two coins, but that’s not very useful. To find the total value of one quarter plus one dime, you change them to the same kind of unit—cents. One quarter equa

In [70]:
openstax_text = prompt_manager.recent_slot_fill_dict[1]["openstax_subsection_texts"]
print(openstax_text)

Find the Least Common Denominator: In the previous section, we explained how to add and subtract fractions with a common denominator. But how can we add and subtract fractions with unlike denominators?  Let’s think about coins again. Can you add one quarter and one dime? You could say there are two coins, but that’s not very useful. To find the total value of one quarter plus one dime, you change them to the same kind of unit—cents. One quarter equals 25 cents and one dime equals 10 cents, so the sum is 35 cents. See Figure 4.7.  Similarly, when we add fractions with different denominators we have to convert them to equivalent fractions with a common denominator. With the coins, when we convert to cents, the denominator is 100. Since there are 100 cents in one dollar, 25 cents is 25100 and 10 cents is 10100. So we add 25100+10100 to get 35100, which is 35 cents.  You have practiced adding and subtracting fractions with common denominators. Now let’s see what you need to do with fractio

In [118]:
# Category "Mn" are the "Nonspacing marks"
# import unicodedata
#''.join(c for c in unicodedata.normalize('NFKD', openstax_text) if unicodedata.category(c) != 'Mn')

In [122]:
tokens = tokenizer.encode(openstax_text)
print(len(tokens))
tokens = [
    token
    for token in tokens
    if token not in stopword_tokens
    and re.fullmatch(
        "[ A-Za-z]+",
        tokenizer.decode(
            [
                token,
            ]
        ),
    )
]
print(len(tokens))

1266
303


In [123]:
", ".join(
    [
        tokenizer.decode(
            [
                token,
            ]
        )
        for token in tokens
    ]
)

' Den, ominator,  explained,  add,  subtract,  fractions,  denominator,  add,  subtract,  fractions,  unlike,  denomin, ators,  coins,  add,  quarter,  dime,  coins,  useful,  quarter,  plus,  dime, cents,  quarter,  equals,  cents,  dime,  equals,  cents,  sum,  cents,  Similarly,  add,  fractions,  denomin, ators,  convert,  equivalent,  fractions,  denominator,  coins,  convert,  cents,  denominator,  cents,  dollar,  cents,  cents,  add,  cents,  practiced,  adding,  subtract, ing,  fractions,  denomin, ators,  fractions,  denomin, ators,  fraction,  tiles,  finding,  denominator,  tile,  tile,  fraction,  tile,  exactly,  pieces,  exactly,  exactly,  pieces,  exactly,  cover,  pieces,  exactly,  cover,  exactly,  cover,  pieces,  smaller,  tiles,  exactly,  cover,  denominator,  largest,  covers,  fractions,  denominator, LCD,  fractions,  denominator,  Notice,  tiles,  cover,  denomin, ators,  multiples,  denomin, ators,  multiple, LC,  denomin, ators,  denominator, LCD,  fractio

In [124]:
for token, count in Counter(tokens).most_common(20):
    print(f"{token} {tokenizer.decode([token,])} {count}")

65995  fractions 26
48012  denominator 19
3046 ators 17
62265  denomin 13
19983  fraction 12
13890  equivalent 10
22745  LCD 10
33356  subtract 9
7041  exactly 9
23534  Fra 9
5247 ctions 9
923  add 8
31291  cents 8
64633  numerator 7
9863  pieces 6
3504  cover 5
9547  factors 5
21130  tiles 4
10190 CM 4
92328  Equivalent 4


In [134]:
def get_logit_bias(
    tokens: list[int],
    min_count: int = 2,
    n_tokens: int | None = None,
    max_tokens: int = 50,
    min_bias: float = 1.0,
    max_bias: float = 5.0,
):
    logit_bias = {}
    c = Counter(tokens).most_common(max_tokens)
    max_count = c[0][1]
    if max_count >= min_count:
        for token, count in c:
            if count < min_count:
                continue
            bias = min_bias + (max_bias - min_bias) * (count / max_count)
            logit_bias[token] = bias
            if n_tokens is not None and len(logit_bias) >= n_tokens:
                break
    return logit_bias


get_logit_bias(tokens)

{65995: 5.0,
 48012: 3.923076923076923,
 3046: 3.6153846153846154,
 62265: 3.0,
 19983: 2.8461538461538463,
 13890: 2.5384615384615383,
 22745: 2.5384615384615383,
 33356: 2.3846153846153846,
 7041: 2.3846153846153846,
 23534: 2.3846153846153846,
 5247: 2.3846153846153846,
 923: 2.230769230769231,
 31291: 2.230769230769231,
 64633: 2.0769230769230766,
 9863: 1.9230769230769231,
 3504: 1.7692307692307692,
 9547: 1.7692307692307692,
 21130: 1.6153846153846154,
 10190: 1.6153846153846154,
 92328: 1.6153846153846154,
 19289: 1.4615384615384617,
 8502: 1.4615384615384617,
 74953: 1.4615384615384617,
 5625: 1.4615384615384617,
 7999: 1.4615384615384617,
 287: 1.4615384615384617,
 10228: 1.4615384615384617,
 8015: 1.4615384615384617,
 7504: 1.4615384615384617,
 31370: 1.4615384615384617,
 9973: 1.3076923076923077,
 17239: 1.3076923076923077,
 44664: 1.3076923076923077,
 9455: 1.3076923076923077,
 65103: 1.3076923076923077,
 25773: 1.3076923076923077,
 5602: 1.3076923076923077,
 31716: 1.30769

In [135]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    request_timeout=20,
    logit_bias=logit_bias,
)
assistant_message = completion["choices"][0]["message"]["content"]
print(assistant_message)

When using PEMDAS with fractions, you follow the same order of operations. However, you may need to convert fractions to equivalent fractions with a common denominator before performing any operations. The common denominator helps ensure that the fractions have the same denominator, making it easier to add, subtract, multiply, or divide them.

To find the least common denominator (LCD), which is the denominator that fractions need to have in common, you can use the least common multiple (LCM) of their denominators. The LCM is the smallest multiple that two numbers have in common. Once you find the LCD, you convert the fractions to equivalent fractions with the LCD as the denominator.

For example, let's say you have the fractions 1/2 and 1/3. The denominators are different, so we find the LCD. The LCM of 2 and 3 is 6. We convert 1/2 to an equivalent fraction with denominator 6 by multiplying both the numerator and denominator by 3: (1/2) x (3/3) = 3/6. Similarly, we convert 1/3 to an e

### Generate responses

In [11]:
ds = []

In [12]:
eval_dir = data_dir / "derived" / "eval"
eval_dir.mkdir(exist_ok=True)
response_filepath = eval_dir / "generations_20230809.pkl"
if response_filepath.exists():
    ds = pickle.load(infile)
len(ds)

0

In [14]:
# intro_prompts_key = "general_math_qa_intro"
intro_prompts_key = "retrieval_reliant_math_qa_intro"
for user_query in tqdm(student_queries, desc="Generating responses"):
    for retrieval_strategy_name, retrieval_strategy in [
        ("math", math_strategy),
        ("recipe", nomath_strategy),
        ("none", noretrieval_strategy),
    ]:
        already_generated = False
        for d in ds:
            if (
                d["user_query"] == user_query
                and d["retrieval_strategy"] == retrieval_strategy_name
                and d["intro_prompts_key"] == intro_prompts_key
            ):
                already_generated = True
        if already_generated:
            continue
        intro_prompt_messages = mathqa_prompts.intro_prompts[intro_prompts_key]["messages"]
        prompt_manager = prompt_utils.PromptManager()
        prompt_manager.set_intro_messages(intro_prompt_messages)
        prompt_manager.set_retrieval_strategy(retrieval_strategy)
        messages = prompt_manager.build_query(user_query)
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
            request_timeout=5,
        )
        assistant_message = completion["choices"][0]["message"]["content"]
        ds.append(
            {
                "user_query": user_query,
                "retrieval_strategy": retrieval_strategy_name,
                "intro_prompts_key": intro_prompts_key,
                "response": assistant_message,
                "messages": messages,
            }
        )
        time.sleep(0.4)  # being a bit polite on repeated api calls

Generating responses: 100%|████████████████████████████████████████████████████████████████████████████████████████| 51/51 [03:49<00:00,  4.50s/it]


In [15]:
len(ds)

153

In [16]:
with open(response_filepath, "wb") as outfile:
    pickle.dump(ds, outfile)

In [17]:
# load generated responses
with open(response_filepath, "rb") as infile:
    pickle.load(infile)

In [21]:
df = (
    pd.DataFrame(ds, columns=["user_query", "retrieval_strategy", "response"])
    .reset_index()
    .rename(columns={"index": "response_id"})
)
df.shape

(153, 4)

In [22]:
df.sample(n=3)

,response_id,user_query,retrieval_strategy,response
148,148,i need help on how to graph quadratic funtions,recipe,Of course! I'm here to help you with graphing ...
99,99,How do you multiply fractions?!?!?,math,"Great question! To multiply fractions, you mul..."
13,13,How do you find the radius?,recipe,"To find the radius of a circle, you can use th..."


In [23]:
df.retrieval_strategy.value_counts()

retrieval_strategy
math      51
recipe    51
none      51
Name: count, dtype: int64

### Construct surveys

Generate two surveys.

 - Randomize query order.
 - Create two questions, comparing recipe vs none and math vs none.
 - Assign every other question to survey 1 or survey 2, alternatingly.

In [30]:
def get_question_with_random_order(group: pd.DataFrame, option_set: list):
    question = {}
    for i, option in enumerate(random.sample(option_set, k=len(option_set))):
        question[f"response{i+1}"] = group[group.retrieval_strategy == option].iloc[0].response
        question[f"response{i+1}_retrieval_strategy"] = option
    return question


def update_curr_survey(curr_survey, n_surveys):
    next_survey = (curr_survey + 1) % n_surveys
    return next_survey


survey = []
n_surveys = 2
curr_survey = 0
for user_query in df.user_query.drop_duplicates().sample(frac=1, random_state=19881):
    group = df[df.user_query == user_query]
    q1 = get_question_with_random_order(group, ["recipe", "none"])
    q2 = get_question_with_random_order(group, ["math", "none"])
    q1 = {"survey": curr_survey, "user_query": user_query, **q1}
    curr_survey = update_curr_survey(curr_survey, n_surveys)
    q2 = {"survey": curr_survey, "user_query": user_query, **q2}
    curr_survey = update_curr_survey(curr_survey, n_surveys)
    survey.append(q1)
    survey.append(q2)
len(survey)

102

In [32]:
eval_dir = data_dir / "derived" / "eval"
eval_dir.mkdir(exist_ok=True)

In [33]:
pd.DataFrame(survey).to_csv(eval_dir / "survey_20230809.csv", index=False)